In [ ]:
import os
import urllib

import earthaccess
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pystac
import pystac_client
import shapely
import stackstac
import torchdata
import xarray as xr
import zen3geo

from torch.utils.data import (  # allows us to load data in batches with multiple workers
    DataLoader,
)

In [ ]:
auth = earthaccess.login(persist=True)  # persist EDL login to netrc file

Rieche 2018 Fiji Lolol Pine Study Area

https://www.mdpi.com/2072-4292/7/5/4973

In [ ]:
from pyproj import Transformer

latitude = -17.578799
longitude = 177.580173

def latlon_to_utm_zone(latitude, longitude):
    zone_number = int((longitude + 180) / 6) + 1
    if latitude < 0:
        hemisphere = 'S'
        epsg_code = 32700 + zone_number
    else:
        hemisphere = 'N'
        epsg_code = 32600 + zone_number
    return zone_number, hemisphere, epsg_code


def latlon_to_utm(lat, lon, epsg_code):
    transformer = Transformer.from_crs("EPSG:4326", f"EPSG:{epsg_code}")
    x, y = transformer.transform(lat, lon)
    return x, y

def create_bbox(x, y, buffer_size):
    x1 = x - buffer_size
    y1 = y - buffer_size
    x2 = x + buffer_size
    y2 = y + buffer_size
    return x1, y1, x2, y2


utm_zone, hemisphere, epsg_code = latlon_to_utm_zone(latitude, longitude)
x, y = latlon_to_utm(latitude, longitude, epsg_code)
buffer_size = 5000  # Change this value to adjust the size of the bounding box
bbox_utm = create_bbox(x, y, buffer_size)

def bbox_to_latlon(epsg_code, x1, y1, x2, y2):
    transformer = Transformer.from_crs(f"EPSG:{epsg_code}", "EPSG:4326", always_xy=True)
    lat1, lon1 = transformer.transform(x1, y1)
    lat2, lon2 = transformer.transform(x2, y2)
    return lat1, lon1, lat2, lon2

bbox_latlon = bbox_to_latlon(epsg_code, *bbox_utm)



In [ ]:
bbox_utm

In [ ]:
bbox_latlon

We select a time range that works for Sentinel-2, post July 2015.

The dates for the original study used Landsat and looked at January 2005-December 2007 for the training period and January 2008 - September 2010 for the Validation period.

In [ ]:
time_range = ["2022-01-01T00:00:00Z", "2023-05-10T23:59:59Z"]
collection_ids = ["HLSL30.v2.0"]  # Harmonized Landsat-8 Sentinel-2 (HLS)

In [ ]:
query = dict(bbox=list(bbox_latlon), datetime=time_range, collections=collection_ids)

In [ ]:
dp = torchdata.datapipes.iter.IterableWrapper(iterable=[query])
dp_pystac_client = dp.search_for_pystac_item(
    catalog_url="https://cmr.earthdata.nasa.gov/stac/LPCLOUD",
)

In [ ]:
def get_all_items(item_search) -> pystac.ItemCollection:
    return item_search.items()

In [ ]:
dp_hls_items = dp_pystac_client.flatmap(fn=get_all_items)
dp_hls_items

In [ ]:
it = iter(dp_hls_items)
item = next(it)
print(item)

In [ ]:
def cloud_cover_filter(item: pystac.Item, threshold=20) -> bool:
    """
    Return True if less than or equal to 20% cloud cover, else False.
    """
    return item.properties["eo:cloud_cover"] <= threshold


In [ ]:
dp_hls_filtered = dp_hls_items.filter(filter_fn=cloud_cover_filter)

In [ ]:
gdal_env = stackstac.DEFAULT_GDAL_ENV.updated(
    always=dict(
        GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
        GDAL_HTTP_MERGE_CONSECUTIVE_RANGES="YES",
        GDAL_HTTP_COOKIEFILE=os.path.expanduser("~/cookies.txt"),
        GDAL_HTTP_COOKIEJAR=os.path.expanduser("~/cookies.txt"),
    )
)
dp_hls_stack = dp_hls_filtered.stack_stac_items(
    assets=["B04", "B03", "B02"],  # RGB bands
    epsg=32611,  # UTM Zone 11N
    resolution=30,  # Spatial resolution of 30 metres
    bounds_latlon=bbox_latlon,
    xy_coords="center",  # pixel centroid coords instead of topleft corner
    dtype=np.float16,  # Use a lightweight data type
    # https://github.com/gjoseph92/stackstac/issues/133
    gdal_env=gdal_env,
)
dp_hls_stack

In [ ]:
it = iter(dp_hls_stack)
dataarray = next(it)
print(dataarray)

In [ ]:
dataarray.isel(time=0).plot.imshow(rgb="band", robust=True)

In [ ]:
torchdata.datapipes.utils.to_graph(dp=dp_hls_stack)

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rx

from pybayts.bayts import bayts_da_to_date_array
from pybayts.bayts import create_bayts_ts
from pybayts.bayts import deseason_ts
from pybayts.bayts import loop_bayts_update
from pybayts.bayts import merge_cpnf_tseries
from pybayts.data.io import read_and_stack_example_tifs


vv_folder = "../tests/baytsdata/s1vv_tseries/"
ndvi_folder = "../tests/baytsdata/lndvi_tseries/"
pdf_type_l = ("gaussian", "gaussian")
chi = .9
cpnf_min = .5
pdf_forest_l = (0, 0.1)  # mean and sd
pdf_nonforest_l = (-0.5, 0.125)  # mean and sd
bwf_l = (0.1, 0.9)
pdf_type_s = ("gaussian", "gaussian")
pdf_forest_s = (-1, 0.75)  # mean and sd
pdf_nonforest_s = (-4, 1)  # mean and sd
bwf_s = (0.1, 0.9)


In [ ]:

latitude = -17.32
longitude = 177.37
utm_zone = 60
x, y = latlon_to_utm(latitude, longitude, epsg_code)
print(x, y)


In [ ]:
from pyproj import Transformer

def convert_utm_to_latlon(utm_zones, x1, y1, x2, y2):
    bbox_coordinates = []
    for zone in utm_zones:
        epsg_code = 32600 - zone  # 32700 is the EPSG code for WGS84 UTM zone 1S, subtract the zone number to get the correct EPSG code
        transformer = Transformer.from_crs(f"EPSG:{epsg_code}", "EPSG:4326")
        lat1, lon1 = transformer.transform(x1, y1)
        lat2, lon2 = transformer.transform(x2, y2)
        bbox_coordinates.append(((lat1, lon1), (lat2, lon2)))
    return bbox_coordinates

x1, y1 = 594_000, 8_000_500
x2, y2 = 598_000, 8_002_500
bbox_coordinates = convert_utm_to_latlon(fiji_zones, x1, y1, x2, y2)
print(bbox_coordinates)


In [ ]:
bbox_coordinates[0]

In [ ]:
from shapely.geometry import box

def get_bbox_centroids(bbox_coordinates):
    centroids = []
    for (lat1, lon1), (lat2, lon2) in bbox_coordinates:
        bbox = box(lat1, lon1, lat2, lon2)
        centroid = bbox.centroid
        centroids.append((centroid.y, centroid.x))
    return centroids

centroids = get_bbox_centroids(bbox_coordinates)
print(centroids)


In [ ]:

s1vv_ts = read_and_stack_example_tifs(vv_folder, ds="vv")
s1vv_ts.name = "s1vv"

lndvi_ts = read_and_stack_example_tifs(ndvi_folder, ds="lndvi")
lndvi_ts.name = "lndvi"